<a href="https://colab.research.google.com/github/nHunter0/Resistor-Detector-/blob/main/Resisitor_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing resitor dataset from kaggle
from google.colab import files
files.upload() #uploading kaggle.json

#changing permissions on kaggle.json
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# downloading, ziping, unziping and removing zip
! kaggle datasets download -d eralpozcan/resistor-dataset
! mkdir train
! unzip \*.zip -d train
! rm *.zip

#test script
'''
import os
dir_path = './train'

files = os.listdir(dir_path)

for file in files:
  print(file)
'''

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [ ]:
def load_data(data_dir, img_height=64, img_width=64, split=0.2):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=split)

    train_generator = datagen.flow_from_directory(
        directory=data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        subset='training',
        seed=123
    )

    validation_generator = datagen.flow_from_directory(
        directory=data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        seed=123
    )

    return train_generator, validation_generator

In [ ]:
train_data, test_data = load_data('train')
class_names = list(train_data.class_indices.keys())
test_images, test_labels = next(train_data)  # get one batch of images and labels

#test script

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i])
    label_index = np.argmax(test_labels[i])  # find index of the class label
    plt.xlabel(class_names[label_index])
plt.show()


In [ ]:
# Define the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add a flatten layer
model.add(Flatten())

# Add dense layers
model.add(Dense(64, activation='relu'))
model.add(Dense(37, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model
model.summary()

# Train the model
history = model.fit(train_data,
                    epochs=50,
                    validation_data=test_data)


In [ ]:
#Saving modle
from google.colab import files
#model.save('mymodle.h5')
files.download('mymodle.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Modle evaluation
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f'Test acc: {test_acc}')
print(f'Test loss: {test_loss}')

# Plots
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the image
img_height = 64
img_width = 64

img = image.load_img('test-image1.png', target_size=(img_height, img_width))

# Preprocess the image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Use the model to predict the class
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions[0])

# Print the predicted class
print(f'The model predicts that this is a {class_names[predicted_class]} resistor.')


1/1 [==============================] - 0s 19ms/step
The model predicts that this is a 470R_1W resistor.
